## Ignore  non  harmful  warnings

In [ ]:
from warnings  import filterwarnings
filterwarnings('ignore')


In [ ]:
import pandas as  pd
import numpy as  np
import seaborn as  sns
import matplotlib.pyplot as  plt

In [ ]:
df  = pd.read_csv('/content/train_machine.csv')

# Basic Checks

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df.rename(columns  =  {'Air temperature [K]' : 'Air temperature',
          'Process temperature [K]':'Process temperature' ,
          'Rotational speed [rpm]' :  'Rotational speed',
               'Torque [Nm]' :'Torque'    ,
                'Tool wear [min]':'Tool wear' } ,  inplace  =  True)

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
df.duplicated().sum()

In [ ]:
df.columns

## seperate x  & y

In [ ]:
x  =  df.drop(['id' , 'Product ID' , 'Machine failure'] , axis  =  1)
y = df['Machine failure' ]

In [ ]:
x.columns

#### Seperate  cat and  numerical  data

In [ ]:
cat  =  x.columns[x.dtypes  =='object']
num =  x.columns[x.dtypes  !='object']

In [ ]:
print(cat)
print()
print()
print(num)

In [ ]:
x.Type.value_counts()

In [ ]:
x.Type.insull().sum()

### create  pipeline

In [ ]:
from sklearn.pipeline  import Pipeline
from sklearn.preprocessing import OrdinalEncoder ,  StandardScaler ,  OneHotEncoder
from  sklearn.impute  import SimpleImputer
from  sklearn.compose  import ColumnTransformer

In [ ]:
cat_pipeline  =  Pipeline(steps  =  [('imputer' ,  SimpleImputer(strategy= 'most_frequent')) ,
                                     ('encoder' , OrdinalEncoder())])
num_pipeline  =  Pipeline(steps  = [('imputer' ,  SimpleImputer(strategy  =  'mean')) ,
                                          ('scaler' , StandardScaler())])

In [ ]:
#  compine  pipelines
pre  =  ColumnTransformer([('cat' ,  cat_pipeline  , cat)  ,
                           ('num' ,  num_pipeline ,  num )])

In [ ]:
pre

### Data transformation

In [ ]:
x_scaled  =  pre.fit_transform(x)

In [ ]:
x_scaled =  pd.DataFrame(x_scaled  ,  columns  =  pre.get_feature_names_out())

In [ ]:
x_scaled.head()

In [ ]:
column  =  []
for i  in  pre.get_feature_names_out():
  column.append(i.split('__')[-1])

In [ ]:
x_scaled.columns =  column

In [ ]:
x_scaled.head(1)

In [ ]:
y.value_counts()

In [ ]:
a  = y.value_counts()
a.plot(kind  =  'bar')

# select features

In [ ]:
from sklearn.feature_selection import mutual_info_classif
mi  =  mutual_info_classif( X  =  x_scaled ,  y  =  y  ,  n_jobs  =  -1 , random_state  = 3 )


In [ ]:
mi

In [ ]:
mi  =  pd.Series(mi)

In [ ]:
mi.index  =  x_scaled.columns

In [ ]:
mi  = mi.sort_values(ascending = False)

In [ ]:
mi.plot(kind  = 'bar')

In [ ]:
mi.index

In [ ]:
col  =  ['HDF', 'Torque', 'Rotational speed', 'OSF', 'Type', 'PWF', 'TWF',
       'Tool wear', 'Air temperature', 'Process temperature']

## final data selection  for  model fitting

In [ ]:
x1  =  x[col]

## Create  pipeline 2


In [ ]:
cat1 =  cat
num1 =  ['Air temperature', 'Process temperature', 'Rotational speed', 'Torque',
       'Tool wear', 'TWF', 'HDF', 'PWF', 'OSF']

In [ ]:
cat_pipeline2  =  Pipeline(steps  =  [('imputer' ,  SimpleImputer(strategy= 'most_frequent')) ,
                                     ('encoder' ,OneHotEncoder(handle_unknown='ignore'))])
num_pipeline2  =  Pipeline(steps  = [('imputer' ,  SimpleImputer(strategy  =  'mean')) ,
                                          ('scaler' , StandardScaler())])

In [ ]:
#  compine  pipelines
pre2  =  ColumnTransformer([('cat' ,  cat_pipeline2  , cat1 ) ,
                           ('num' ,  num_pipeline2 ,  num1)])

In [ ]:
pre2

In [ ]:
final_x  =  pre2.fit_transform(x1)
final_x =  pd.DataFrame(final_x  ,  columns  =  pre2.get_feature_names_out())

In [ ]:
final_x.head()

In [ ]:
final_x.shape

### train test  split

In [ ]:
from  sklearn.model_selection import train_test_split ,  GridSearchCV

In [ ]:
xtrain  , xtest  ,  ytrain  , ytest  = train_test_split(final_x  ,  y  , test_size   =  0.2  ,  random_state  =  3)

In [ ]:
print(xtrain.shape  , ytrain.shape)
print(xtest.shape  ,  ytest.shape)

### Model  fitting and  evaluation

In [ ]:
from  sklearn.linear_model import  LogisticRegression
from sklearn.tree  import  DecisionTreeClassifier
from  sklearn.ensemble  import RandomForestClassifier , GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from  sklearn.svm  import SVC
import xgboost
from xgboost  import XGBClassifier

In [ ]:
data  =  {'LR'  :  LogisticRegression(),
          "DT" : DecisionTreeClassifier(max_depth  =  5 ,  min_samples_split=10 ),
          "RF" : RandomForestClassifier(max_depth  =  5 ,  min_samples_split=10 ) ,
          "GB" : GradientBoostingClassifier( ) ,
          'XGB' : XGBClassifier() ,
          "KNN" : KNeighborsClassifier(n_neighbors= 7) ,
          "SVM" : SVC()}

## Create  function for evaluation

In [ ]:
from  sklearn.metrics  import f1_score  ,  classification_report , confusion_matrix

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
def evaluation(x_train  ,  y_train ,  x_test ,  y_test):
  data  =  {'LR'  :  LogisticRegression(),
          "DT" : DecisionTreeClassifier(max_depth  =  5 ,  min_samples_split=10 ),
          "RF" : RandomForestClassifier(max_depth  =  5 ,  min_samples_split=10 ) ,
          "GB" : GradientBoostingClassifier( ) ,
          'XGB' : XGBClassifier() ,
          "KNN" : KNeighborsClassifier(n_neighbors= 7) ,
          "SVM" : SVC()}
  for  name  ,  model in  data.items():
    print(model)

In [ ]:
evaluation(xtrain  , ytrain ,xtest , ytest)

In [ ]:
LogisticRegression().fit(xtrain  ,  ytrain)

In [85]:
def  Evaluation( x_train  , y_train  , x_test  ,  y_test):

  data  =  {'LR'  :  LogisticRegression(),
          "DT" : DecisionTreeClassifier(max_depth  =  5 ,  min_samples_split=10 ),
          "RF" : RandomForestClassifier(max_depth  =  5 ,  min_samples_split=10 ) ,
          "GB" : GradientBoostingClassifier( ) ,
          'XGB' : XGBClassifier() ,
          "KNN" : KNeighborsClassifier(n_neighbors= 7) ,
          "SVM" : SVC()}
  for  name  ,  model in  data.items():
    model.fit(x_train ,  y_train  )
    train_pred  =  model.predict(x_train)
    test_pred  =  model.predict(x_test)
    f1_tr  =  f1_score(y_train  ,  train_pred)
    f1_ts  =  f1_score(y_test  ,  test_pred)
    cv  = cross_val_score( estimator  =  model , X  =  x_train  ,  y  = y_train  ,  cv  =  5  ,  n_jobs= -1)
    CR_tr  =  classification_report(y_train  , train_pred  , )
    cf_tr  =  confusion_matrix(y_train  ,  train_pred)
    CR_test  =  classification_report(y_test  ,  test_pred)
    cf_test  =  confusion_matrix(y_test  , test_pred)

    print(f"{name}")
    print(f"Train f1  --> {f1_tr}")
    print(f"Test f1  -- > {f1_ts}")
    print(f"Cv  -->  {cv}")
    print(cv.mean())
    print(f"cr_tr  -- >  {CR_tr}")
    print(f"cf_tr --> {cf_tr}")
    print(f"CR_test  --> {CR_test}")
    print(f"cf_test --> {cf_test}")
    print()
    print('-------------------------------------------------')
    print()




In [86]:
Evaluation( xtrain  ,  ytrain  ,  xtest  ,  ytest)

LR
Train f1  --> 0.8631303208906352
Test f1  -- > 0.8624833110814419
Cv  -->  [0.99647258 0.99633515 0.99615191 0.99555617 0.99633498]
0.9961701579655411
cr_tr  -- >                precision    recall  f1-score   support

           0       1.00      1.00      1.00    107418
           1       0.99      0.76      0.86      1725

    accuracy                           1.00    109143
   macro avg       0.99      0.88      0.93    109143
weighted avg       1.00      1.00      1.00    109143

cf_tr --> [[107407     11]
 [   407   1318]]
CR_test  -->               precision    recall  f1-score   support

           0       1.00      1.00      1.00     26863
           1       0.99      0.76      0.86       423

    accuracy                           1.00     27286
   macro avg       0.99      0.88      0.93     27286
weighted avg       1.00      1.00      1.00     27286

cf_test --> [[26860     3]
 [  100   323]]

-------------------------------------------------

DT
Train f1  --> 0.8639790

### Gradient Boosting is giving best results
--  use  gradient boosting  for  further  prediction

In [87]:
gb  =  GradientBoostingClassifier()

In [89]:
#  import data

In [90]:
path  =  '/content/test_machine.csv'

In [91]:
pre2

ColumnTransformer(transformers=[('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoder',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 Index(['Type'], dtype='object')),
                                ('num',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler', StandardScaler())]),
                                 ['Air temperature', 'Process temperature',
                                  'Rotational speed', 'Torque', 'Tool wear',
                                  'TWF', 'HDF', 'PWF', 'OSF'])])

In [92]:
sample  =  pd.read_csv(path)

In [93]:
sample.head()

,id,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],TWF,HDF,PWF,OSF,RNF
0,136429,L50896,L,302.3,311.5,1499.0,38.0,60.0,0.0,0.0,0.0,0.0,0.0
1,136430,L53866,L,301.7,311.0,1713.0,28.8,17.0,0.0,0.0,0.0,0.0,0.0
2,136431,L50498,L,301.3,310.4,1525.0,37.7,96.0,0.0,0.0,0.0,0.0,0.0
3,136432,M21232,M,300.1,309.6,1479.0,47.6,5.0,0.0,0.0,0.0,0.0,0.0
4,136433,M19751,M,303.4,312.3,1515.0,41.3,114.0,0.0,0.0,0.0,0.0,0.0


In [94]:
sample.shape

(81520, 13)

In [97]:
pre2.get_feature_names_out()

array(['cat__Type_H', 'cat__Type_L', 'cat__Type_M',
       'num__Air temperature', 'num__Process temperature',
       'num__Rotational speed', 'num__Torque', 'num__Tool wear',
       'num__TWF', 'num__HDF', 'num__PWF', 'num__OSF'], dtype=object)

In [99]:
sample.columns =  ['id', 'Product ID', 'Type', 'Air temperature',
       'Process temperature', 'Rotational speed', 'Torque',
       'Tool wear', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF']

In [100]:
sample_scaled  =  pre2.transform(sample)

In [101]:
sample_scaled  =  pd.DataFrame(sample_scaled ,  columns  =  pre2.get_feature_names_out())

In [102]:
sample_scaled

,cat__Type_H,cat__Type_L,cat__Type_M,num__Air temperature,num__Process temperature,num__Rotational speed,num__Torque,num__Tool wear,num__TWF,num__HDF,num__PWF,num__OSF
0,0.0,1.0,0.0,1.308759,1.125444,-0.153753,-0.276240,-0.694271,-0.03945,-0.072021,-0.049016,-0.063038
1,0.0,1.0,0.0,0.986567,0.764477,1.388744,-1.358313,-1.366516,-0.03945,-0.072021,-0.049016,-0.063038
2,0.0,1.0,0.0,0.771772,0.331317,0.033653,-0.311525,-0.131461,-0.03945,-0.072021,-0.049016,-0.063038
3,0.0,0.0,1.0,0.127386,-0.246230,-0.297912,0.852880,-1.554119,-0.03945,-0.072021,-0.049016,-0.063038
4,0.0,0.0,1.0,1.899446,1.702992,-0.038426,0.111895,0.149943,-0.03945,-0.072021,-0.049016,-0.063038
...,...,...,...,...,...,...,...,...,...,...,...,...
81515,0.0,0.0,1.0,1.523554,1.630798,-0.802467,1.523295,0.790921,-0.03945,-0.072021,-0.049016,-0.063038
81516,0.0,0.0,1.0,0.556977,0.981058,-0.478110,0.594124,-0.068927,-0.03945,-0.072021,-0.049016,-0.063038
81517,0.0,1.0,0.0,-0.302204,-0.823778,-0.932209,1.205730,1.510066,-0.03945,-0.072021,-0.049016,-0.063038
81518,0.0,1.0,0.0,-1.590974,-1.329132,-0.413239,-0.264478,-0.522301,-0.03945,-0.072021,-0.049016,-0.063038


In [104]:
gb.fit(xtrain  ,  ytrain)

GradientBoostingClassifier()

In [105]:
gb.score(xtrain  ,  ytrain)

0.9965091668728182

In [106]:
gb.score(xtest  , ytest)

0.9961518727552591

In [107]:
train_pred  =  gb.predict(xtrain)
test_pred  =  gb.predict(xtest)

In [108]:
f1_score(ytrain  ,  train_pred)

0.8768186226964112

In [109]:
f1_score(ytest  ,  test_pred)

0.8609271523178808

In [110]:
predict  =  gb.predict(sample_scaled)

In [112]:
len(predict)

81520